In [1]:
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pip install netCDF4

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Gustavi\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Gustavi\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [7]:
#FILE = './training/*.nc'
#ds = xr.open_mfdataset(FILE)
files = ["./training/20101101.nc", "./training/20101102.nc", "./training/20101107.nc", "./training/20101108.nc",
        "./training/20101109.nc", "./training/20101115.nc", "./training/20101116.nc", "./training/20101118.nc",
        "./training/20101119.nc", "./training/20101121.nc", "./training/20101122.nc", "./training/20101126.nc"]
ds = xr.open_mfdataset(files)
ds

<xarray.Dataset>
Dimensions:                         (time: 3070, x: 480, y: 480)
Coordinates:
  * x                               (x) int32 549850 550350 ... 788850 789350
  * y                               (y) int32 5271250 5270750 ... 5031750
  * time                            (time) datetime64[ns] 2010-11-01 ... 2010...
Data variables:
    transverse_mercator             (time) |S1 b'' b'' b'' b'' ... b'' b'' b''
    equivalent_reflectivity_factor  (time, y, x) float32 dask.array<chunksize=(286, 480, 480), meta=np.ndarray>
Attributes:
    Conventions:         CF-1.7
    GDAL_AREA_OR_POINT:  Area
    title:               MAX(Z) Reflectivity day 2010-11-01
    institution:         Meteotrentino, Fondazione Bruno Kessler
    source:              radar observation mt.Macaion
    site_lat_lon_elev:   46.494002, 11.208826, 1876
    references:          original dataset https://doi.org/10.5281/zenodo.3577451
    comments:            NetCDF generation code https://github.com/MPBA/TAASR...
    chunks:              164 122
    tags:                rain snow

In [8]:
reflectivity_ds = ds.equivalent_reflectivity_factor  

In [9]:
def window_generator_xarray(dataset, n_steps):
    x, y = list(), list()
    for i in range(len(dataset)):
        end_ix = i + n_steps
        end_iy = end_ix + n_steps

        x_data = dataset.isel(time=slice(i,end_ix))
        x_data = x_data.expand_dims(dim='sample', axis=0)
        y_data = dataset.isel(time=slice(end_ix,end_iy))
        y_data = y_data.expand_dims(dim='sample', axis=0)

        if y_data.time.size < n_steps:
            break
        
        x_data = x_data.drop(labels='time')
        x.append(x_data)
        y_data = y_data.drop(labels='time')
        y.append(y_data)

    return x, y

In [10]:
x_list, y_list = window_generator_xarray(reflectivity_ds, n_steps=5)
x_data = xr.concat(x_list, dim='sample')
x_data = x_data.expand_dims(dim='channel', axis=-1)

y_data = xr.concat(y_list, dim='sample')
y_data = y_data.expand_dims(dim='channel', axis=-1)

print('Sample size with X: ', x_data.sample.size)
print('Sample size with y: ', y_data.sample.size)

Sample size with X:  3061
Sample size with y:  3061


In [11]:
x_data

,Array,Chunk
Bytes,13.14 GiB,900.00 kiB
Shape,"(3061, 5, 480, 480, 1)","(1, 1, 480, 480, 1)"
Count,82749 Tasks,15305 Chunks
Type,float32,numpy.ndarray


In [16]:
x_data.shape

(3061, 5, 480, 480, 1)

In [19]:
x_data.transpose("sample","channel","time","x","y")

,Array,Chunk
Bytes,13.14 GiB,900.00 kiB
Shape,"(3061, 1, 5, 480, 480)","(1, 1, 1, 480, 480)"
Count,98054 Tasks,15305 Chunks
Type,float32,numpy.ndarray
